# Small Demo for FAS jsonl File Structure from Twarc

This is just a short ipynb to record what the structure of a retrieved jsonl object is from twarc.

First print the the dict structure of a jsonl object.

In [2]:
import jsonlines
import glob

path = '../../data/01_raw/01_group'
user_jsonl_file = glob.glob(path+'/timeline*.jsonl')[0]

def print_keys(dic, shift = 0):
    for key, value in dic.items():
        typ = str(type(value)).split('\'')[-2]
        print(' '*shift + key + ' ('+ typ +')')
        if isinstance(value, dict):
            shift += 2
            print_keys(value, shift=shift)
            shift -= 2

with jsonlines.open(user_jsonl_file) as reader:
    for tweet_jsonl in reader:
        print_keys(tweet_jsonl)
        break

data (list)
includes (dict)
  users (list)
  tweets (list)
  media (list)
errors (list)
meta (dict)
  newest_id (str)
  oldest_id (str)
  result_count (int)
  next_token (str)
__twarc (dict)
  url (str)
  version (str)
  retrieved_at (str)


In [19]:
# for m3inference
import re
import tqdm
for i in tqdm.tqdm(glob.glob(path+'/timeline*.jsonl')[:1]):
    current_user = re.split('[_.]', i)[-2]
    with jsonlines.open(i, 'r') as f:
        for line in f:
            # # user_in_file = line['includes']['users'][0]['id']
            # print(line['includes']['users'][0]['profile_image_url'])
            for x in line['data']:
                if x['referenced_tweets'][0]['type'] == 'quoted':
                    print(x)
                    # print(x['referenced_tweets'][0])
                break

    # if current_user!=user_in_file:
        # print(current_user, user_in_file)

  0%|          | 0/1 [00:00<?, ?it/s]

{'public_metrics': {'retweet_count': 2, 'reply_count': 3, 'like_count': 8, 'quote_count': 1}, 'entities': {'urls': [{'start': 21, 'end': 44, 'url': 'https://t.co/AVCcVeOfde', 'expanded_url': 'https://twitter.com/denverpost/status/987411881588600833', 'display_url': 'twitter.com/denverpost/sta…'}]}, 'source': 'Twitter for Android', 'reply_settings': 'everyone', 'possibly_sensitive': True, 'author_id': '2740546270', 'created_at': '2018-04-20T20:02:19.000Z', 'lang': 'en', 'id': '987421222517473281', 'conversation_id': '987421222517473281', 'referenced_tweets': [{'type': 'quoted', 'id': '987411881588600833'}], 'text': 'What. Fresh. Hell. 😡 https://t.co/AVCcVeOfde'}
{'entities': {'annotations': [{'start': 28, 'end': 31, 'probability': 0.6068, 'type': 'Organization', 'normalized_text': 'dems'}, {'start': 52, 'end': 62, 'probability': 0.9489, 'type': 'Organization', 'normalized_text': 'REPUBLICANS'}, {'start': 115, 'end': 123, 'probability': 0.5963, 'type': 'Person', 'normalized_text': 'Ameri

  0%|          | 0/1 [00:00<?, ?it/s]


KeyError: 'referenced_tweets'

Of the top 4 in the hierarchy meta is only really useful to continue collecting tweets with the next_token value, and _twarc has some information about the twarc package.

data contains the core of the results from the Full Archive Search query, whereas includes brings referenced tweets and authors (users) of those tweets. We show this below

In [2]:
with jsonlines.open(user_jsonl_file) as reader:
    for tweet_jsonl in reader:
        # print(user_jsonl_file)
        # print('\n')
        # print(len(tweet_jsonl['data']))
        # print('\n')
        # print(tweet_jsonl['data'][0])
        # print('\n')
        # print(tweet_jsonl['includes']['users'][1])
        # print('\n')
        # print(len(tweet_jsonl['includes']['users']))
        # print('\n')
        # print(len(tweet_jsonl['includes']['tweets']))
        # print('\n')

        print(tweet_jsonl['data'][0].keys())

        ref_ids = []
        for tweet_obj in tweet_jsonl['data']:
            
            if 'referenced_tweets' in tweet_obj:
                for reffed_tweet in tweet_obj['referenced_tweets']:
                    print(reffed_tweet)
                    break
                    ref_ids.append(reffed_tweet['id'])

        for tweet_obj in tweet_jsonl['includes']['tweets']:
            assert tweet_obj['id'] in ref_ids

            # remove ids from the list as you go along so each tweet in 'includes' must have a corresponding entry.
            ref_ids.remove(tweet_obj['id'])
        print('All OK')

        break

dict_keys(['public_metrics', 'created_at', 'referenced_tweets', 'id', 'possibly_sensitive', 'reply_settings', 'author_id', 'lang', 'conversation_id', 'entities', 'source', 'text'])
{'type': 'retweeted', 'id': '922392644084383744'}
{'type': 'retweeted', 'id': '922434649841741825'}
{'type': 'retweeted', 'id': '922472700412645377'}
{'type': 'retweeted', 'id': '922517890166697984'}
{'type': 'retweeted', 'id': '922517749565288448'}
{'type': 'retweeted', 'id': '922451846949515264'}
{'type': 'retweeted', 'id': '922468531173703680'}
{'type': 'retweeted', 'id': '922506087252635648'}
{'type': 'retweeted', 'id': '922100750393647104'}
{'type': 'retweeted', 'id': '922451900619833344'}
{'type': 'retweeted', 'id': '922438548631953408'}
{'type': 'retweeted', 'id': '922413404760948736'}
{'type': 'retweeted', 'id': '922430975581110272'}
{'type': 'retweeted', 'id': '922397259085504512'}
{'type': 'retweeted', 'id': '922481532702375942'}
{'type': 'retweeted', 'id': '922225668837933056'}
{'type': 'retweeted

AssertionError: 